# SeQR_BackEnd

### 1. 단축 URL이 이동시키는 원본 URL 가져오기 (리디렉션 추적)

In [ ]:
# pip install BeautifulSoup

In [ ]:
import requests

def get_final_url(url):
    try:
        # HTTP HEAD 요청을 보내고 리디렉션을 추적
        response = requests.head(url, allow_redirects=True)
        return response.url  # 최종 URL 반환
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}  # 요청 중 발생한 예외 처리

# 사용 예시
url = 'https://bIt.ly/2024먼동제부스신청'
final_url = get_final_url(url)
print(final_url)


https://docs.google.com/forms/d/e/1FAIpQLSchvOglWaMflxy7fc6GyvbJrKfDRGmLu_ORC-lD_AatH7Pb9A/closedform


### 2. 요청 보내고 메타 데이터 받아오기

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_url_preview(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.text, 'html.parser')

        # Open Graph 및 기본 메타 태그에서 정보 추출
        title = soup.find("meta", property="og:title") or soup.find("title")
        description = soup.find("meta", property="og:description") or soup.find("meta", attrs={"name": "description"})
        image = soup.find("meta", property="og:image")

        # 메타 태그가 없을 경우 대비
        preview_info = {
            "title": title["content"] if title and title.has_attr("content") else title.string if title else "No title found",
            "description": description["content"] if description else "No description found",
            "image": image["content"] if image else "No image found"
        }

        return preview_info

    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# 사용 예시
url = "https://ticket.interpark.com/webzine/paper/TPNoticeView.asp?bbsno=34&no=53330"
preview = get_url_preview(url)
print(preview)


{'title': '인터파크 티켓', 'description': 'No description found', 'image': 'No image found'}


### 3. 백엔드 코드

In [ ]:
# pip install fastapi
pip install uvicorn

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from seqr_model import predict_url  # seqr_model에서 predict_url 함수 불러오기
import requests
from bs4 import BeautifulSoup

# FastAPI 애플리케이션 초기화
app = FastAPI()

# 요청 바디를 정의하는 Pydantic 모델
class TextRequest(BaseModel):
    text: str

# 최종 URL을 가져오는 함수
def get_final_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        return response.url
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# URL 미리보기 정보 가져오는 함수
def get_url_preview(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find("meta", property="og:title") or soup.find("title")
        description = soup.find("meta", property="og:description") or soup.find("meta", attrs={"name": "description"})
        image = soup.find("meta", property="og:image")

        preview_info = {
            "title": title["content"] if title and title.has_attr("content") else title.string if title else "No title found",
            "description": description["content"] if description else "No description found",
            "image": image["content"] if image else "No image found"
        }
        return preview_info
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

@app.post("/predict")
async def predict(request: TextRequest):
    # seqr_model.py의 predict_url 함수 사용하여 예측 수행
    predicted_class, probabilities = predict_url(request.text)

    # URL 처리
    urls = [word for word in request.text.split() if word.startswith("http")]
    url_data = []
    for url in urls:
        final_url = get_final_url(url)
        preview_info = get_url_preview(url)
        url_data.append({
            "url": url,
            "final_url": final_url,
            "preview": preview_info
        })

    return {
        "predicted_class": "predicted_class",
        "probabilities": "probabilities",
        "original_text": request.text,
        "url_data": url_data
    }

In [ ]:
!pip install nest_asyncio

### 3. 모델 예측 제외 테스트 코드

In [ ]:
import nest_asyncio
from fastapi import FastAPI
from fastapi.testclient import TestClient
import requests
from bs4 import BeautifulSoup

# Apply the nest_asyncio to allow asyncio to run within Jupyter/Colab
nest_asyncio.apply()

# FastAPI app definition
app = FastAPI()

# Function to get the final URL after redirection
def get_final_url(url):
    try:
        # HTTP HEAD 요청을 보내고 리디렉션을 추적
        response = requests.head(url, allow_redirects=True)
        return response.url  # 최종 URL 반환
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}  # 요청 중 발생한 예외 처리

# Function to extract URL preview (title, description, image)
def get_url_preview(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.text, 'html.parser')

        # Open Graph 및 기본 메타 태그에서 정보 추출
        title = soup.find("meta", property="og:title") or soup.find("title")
        description = soup.find("meta", property="og:description") or soup.find("meta", attrs={"name": "description"})
        image = soup.find("meta", property="og:image")

        # 메타 태그가 없을 경우 대비
        preview_info = {
            "title": title["content"] if title and title.has_attr("content") else title.string if title else "No title found",
            "description": description["content"] if description else "No description found",
            "image": image["content"] if image else "No image found"
        }

        return preview_info

    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

@app.post("/predict")
def predict(data: dict):
    url = data.get("text")
    if not url:
        return {"error": "No URL provided"}

    # Get final URL and URL preview
    final_url = get_final_url(url)
    url_preview = get_url_preview(final_url)

    return {
        "predicted_class": "example_class",
        "probabilities": [0.1, 0.9],
        "original_text": url,
        "final_url": final_url,
        "url_preview": url_preview
    }

# TestClient initialization
client = TestClient(app)

# Test function
def test_predict_endpoint():
    # Test data with a URL
    data = {"text": "https://www.naver.com/"}

    # Sending POST request
    response = client.post("/predict", json=data)

    # Verify the response status code and structure
    assert response.status_code == 200
    response_data = response.json()

    # Assert expected response structure
    assert "predicted_class" in response_data
    assert "probabilities" in response_data
    assert "original_text" in response_data
    assert "final_url" in response_data
    assert "url_preview" in response_data

    # Output the results
    print("Response JSON:", response_data)
    print("Predicted Class:", response_data.get("predicted_class"))
    print("Probabilities:", response_data.get("probabilities"))
    print("Final URL:", response_data.get("final_url"))
    print("URL Preview:", response_data.get("url_preview"))

# Execute test function
test_predict_endpoint()


Response JSON: {'predicted_class': 'example_class', 'probabilities': [0.1, 0.9], 'original_text': 'https://www.naver.com/', 'final_url': 'https://www.naver.com/', 'url_preview': {'title': '네이버', 'description': '네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요', 'image': 'https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png'}}
Predicted Class: example_class
Probabilities: [0.1, 0.9]
Final URL: https://www.naver.com/
URL Preview: {'title': '네이버', 'description': '네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요', 'image': 'https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png'}
